<a href="https://colab.research.google.com/github/minerva79/medt_nlp/blob/main/MIMIC_IV_multi_label_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Feb 19 13:38:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import pandas as pd

# Load the pickle file into a pandas DataFrame
combined_df = pd.read_pickle("/content/drive/MyDrive/combined_df.pkl")

In [ ]:
import pandas as pd

# Assuming combined_df is your DataFrame and 'y' is the column with labels
combined_df = combined_df[combined_df['y'].apply(lambda x: len(x) <= 1)]

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(combined_df, test_size=0.2)


In [ ]:
import torch
from torch.utils.data import TensorDataset

def create_dataset(df):
    input_ids = list(df['input_ids'])
    attention_masks = list(df['attention_mask'])
    # Directly reference the label columns
    label_columns = ['anemia', 'atrial fibrillation', 'hyperlipidemia', 'hypertension', 'pneumonia']
    labels = df[label_columns].values

    return TensorDataset(torch.stack(input_ids), torch.stack(attention_masks), torch.tensor(labels, dtype=torch.float))

train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)

In [ ]:
from torch.utils.data import SubsetRandomSampler
from torch.utils.data import DataLoader
import numpy as np

# Set the batch size
batch_size = 2 # Adjust based on your GPU memory

# Randomly sample indices for a subset of the validation set
subset_indices = np.random.choice(len(val_dataset), size=int(0.5 * len(val_dataset)), replace=False)
sampler = SubsetRandomSampler(subset_indices)

validation_dataloader_sub = DataLoader(val_dataset, batch_size=batch_size, sampler=sampler)

# Randomly sample indices for a subset of the validation set
subset_indices = np.random.choice(len(train_dataset), size=int(0.5 * len(train_dataset)), replace=False)
sampler = SubsetRandomSampler(subset_indices)

train_dataloader_sub = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 4 # Adjust based on your GPU memory

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
validation_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)


In [ ]:
from transformers import LongformerForSequenceClassification

model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=5)
model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LongformerForSequenceClassification(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
          

In [ ]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

Using device:  cuda


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
num_epochs = 3
optimizer = AdamW(model.parameters(), lr=1e-5)  # Learning rate
#optimizer = torch.optim.SGD(model.parameters(), lr=5e-6)

total_steps = len(train_dataloader) * num_epochs  # Number of training epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss
import torch
import numpy as np

def train(model, train_dataloader, validation_dataloader, optimizer, scheduler, accumulation_steps, num_epochs=3):
    for epoch_i in range(num_epochs):
        # Training
        model.train()
        total_train_loss = 0
        optimizer.zero_grad()

        for step, batch in enumerate(train_dataloader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs.loss / accumulation_steps  # Normalize the loss since it is averaged
            total_train_loss += loss.item()

            loss.backward()

            # Perform optimization step every `accumulation_steps` batches
            if (step + 1) % accumulation_steps == 0 or (step + 1) == len(train_dataloader):
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

        avg_train_loss = total_train_loss / len(train_dataloader)
        print(f"Epoch {epoch_i + 1} | Average Training Loss: {avg_train_loss}")

        # Validation
        model.eval()
        all_predictions = []
        all_true_labels = []

        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            with torch.no_grad():
                outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

            # Apply sigmoid and threshold
            probabilities = torch.sigmoid(outputs.logits)
            predictions = (probabilities > 0.5).int()

            # Move predictions and labels to CPU
            predictions = predictions.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            # Store predictions and true labels
            all_predictions.append(predictions)
            all_true_labels.append(label_ids)

        # Convert list of arrays to single numpy arrays
        all_predictions = np.vstack(all_predictions)
        all_true_labels = np.vstack(all_true_labels)

        # Calculate evaluation metrics
        f1 = f1_score(all_true_labels, all_predictions, average='weighted')
        precision = precision_score(all_true_labels, all_predictions, average='weighted')
        recall = recall_score(all_true_labels, all_predictions, average='weighted')
        hammingloss = hamming_loss(all_true_labels, all_predictions)

        print(f"Epoch {epoch_i + 1} | Validation F1-Score: {f1}")
        print(f"Epoch {epoch_i + 1} | Validation Precision: {precision}")
        print(f"Epoch {epoch_i + 1} | Validation Recall: {recall}")
        print(f"Epoch {epoch_i + 1} | Validation Hamming Loss: {hammingloss}")

# Call the train function
train(model, train_dataloader, validation_dataloader, optimizer, scheduler, accumulation_steps=2, num_epochs=1)

Epoch 1 | Average Training Loss: 0.1797786564535151
Epoch 1 | Validation F1-Score: 0.9554680214609808
Epoch 1 | Validation Precision: 0.9580397798560104
Epoch 1 | Validation Recall: 0.9530938123752495
Epoch 1 | Validation Hamming Loss: 0.014833333333333334


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

torch.save(model.state_dict(), '/content/drive/My Drive/model_state_dict.pt')
torch.save(model, '/content/drive/My Drive/entire_model.pt')


In [ ]:
from torch.utils.data import Subset

# Assuming validation_dataloader uses a dataset named val_dataset
num_examples = 10
subset_indices = range(num_examples)
validation_dataloader_subw = DataLoader(Subset(val_dataset, subset_indices), batch_size=batch_size)

model.eval()  # Set the model to evaluation mode

examples_predicted_labels = []
examples_true_labels = []

for batch in validation_dataloader_subw:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    # Apply sigmoid and threshold for multi-label classification
    probabilities = torch.sigmoid(outputs.logits)
    predictions = (probabilities > 0.5).int()

    # Store predictions and labels
    examples_predicted_labels.append(predictions.detach().cpu().numpy())
    examples_true_labels.append(b_labels.detach().cpu().numpy())

    # Break the loop if we've already got 10 examples
    if len(examples_predicted_labels) * batch_size >= num_examples:
        break

# Flatten the lists and slice to get only 10 examples
examples_predicted_labels = np.vstack(examples_predicted_labels)[:num_examples]
examples_true_labels = np.vstack(examples_true_labels)[:num_examples]

# Display the results
print("Displaying 10 examples of true labels and predicted labels:")
for i in range(num_examples):
    print(f"Example {i+1}")
    print(f"True Labels: {examples_true_labels[i]}")
    print(f"Predicted Labels: {examples_predicted_labels[i]}\n")


Displaying 10 examples of true labels and predicted labels:
Example 1
True Labels: [0. 0. 0. 1. 0.]
Predicted Labels: [0 0 0 1 0]

Example 2
True Labels: [0. 0. 0. 1. 0.]
Predicted Labels: [0 0 0 1 0]

Example 3
True Labels: [0. 0. 1. 0. 0.]
Predicted Labels: [0 0 1 0 0]

Example 4
True Labels: [1. 0. 0. 0. 0.]
Predicted Labels: [1 0 0 0 0]

Example 5
True Labels: [0. 0. 0. 0. 0.]
Predicted Labels: [0 0 0 0 0]

Example 6
True Labels: [0. 0. 0. 0. 1.]
Predicted Labels: [0 0 0 0 1]

Example 7
True Labels: [0. 0. 1. 0. 0.]
Predicted Labels: [0 1 0 0 0]

Example 8
True Labels: [0. 0. 0. 1. 0.]
Predicted Labels: [0 0 0 1 0]

Example 9
True Labels: [0. 0. 0. 1. 0.]
Predicted Labels: [0 0 0 1 0]

Example 10
True Labels: [0. 0. 1. 0. 0.]
Predicted Labels: [0 0 1 0 0]



In [ ]:
# Extract the first 5 examples from val_df
examples_text = val_df.iloc[:5]['text']  # Replace 'input_column' with the actual column name
label_columns = ['anemia', 'atrial fibrillation', 'hyperlipidemia', 'hypertension', 'pneumonia']
examples_true_labels = val_df.iloc[:5][label_columns].values

# Create a subset DataLoader for the first 5 examples
subset_indices = range(5)
validation_dataloader_sub = DataLoader(Subset(val_dataset, subset_indices), batch_size=1)

model.eval()  # Set the model to evaluation mode

examples_predicted_labels = []

for batch in validation_dataloader_sub:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)

    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    probabilities = torch.sigmoid(outputs.logits)
    predictions = (probabilities > 0.5).int()

    examples_predicted_labels.append(predictions.detach().cpu().numpy())

# Flatten the predictions list
examples_predicted_labels = np.vstack(examples_predicted_labels)

# Display the results
print("Displaying 5 examples of input text, true labels, and predicted labels:")
for i in range(5):
    print(f"Example {i+1}")
    print(f"Input Text: {examples_text.iloc[i]}")
    print(f"True Labels: {examples_true_labels[i]}")
    print(f"Predicted Labels: {examples_predicted_labels[i]}\n")


Displaying 5 examples of input text, true labels, and predicted labels:
Example 1
Input Text: name unit admission date discharge date date birth sex service medicine allergies ibuprofen nsaids attending chief complaint syncopal episode major surgical invasive procedure none history present illness mr. year-old man idiopathic pulmonary fibrosis emphysema complicated pulmonary hypertension thus requiring home oxygen 4l via nasal cannula transferred icu evaluation management hypoxemia contacted dr. phone report progressive dyspnea exertion resting o2sat 92 4 l desaturation 70 's exertion unable come hospital evaluation time subsequently admitted osh following syncopal episode walking stairs using supplemental oxygen began feel lightheaded top steps walked steps bedroom put arms towards bed lost consciousness wife witnessed bang side face dresser bleeding currently pain next thing remembers paramedics arrived home denies prodromal chest pain palpitations shortness breath nausea although re